# Neo4J Import and Queries

In [1]:
pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 4.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.9.0-py3-none-any.whl size=259467 sha256=7d771a685bc27000c954892ad5dfc871af0224b66a61cca7f61e90a6d6c1d442
  Stored in directory: /home/jovyan/.cache/pip/wheels/5a/6a/b3/89ab0efac04cf204cfc65f2bff4776865cfe5ff2e1b94ebaff
Successfully built neo4j
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import random
import csv

## Aufbau der Datenbankverbindung

In [3]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [4]:
conn = Neo4jConnection(uri="bolt://graph_db:7687", user="neo4j", pwd="password")

In [5]:
conn.query(f"MATCH (n) RETURN n LIMIT 1;")

[]

## Importieren der Knoten und Beziehungen

### Einlesen des "twitter_combined" Files

In [6]:
follows_file = open('twitter_combined.txt', 'r')
lines = follows_file.readlines()

follows = []
users = set()
for line in lines:
    line = line.replace('\n','')
    id1, id2 = line.split(' ')
    users.add(id1)
    users.add(id2)
    follows.append([id1, id2])

### Erstellung des CSV Files für den Import der User

In [7]:
count = 0
f = open("import/users.csv", "w")
f.write("userId,name,:Label\n")
for user in users:
    f.write(user + ",user" + str(count) + ",USER\n")
    count += 1
f.close()

### Importieren der User

In [8]:
%%time
conn.query("LOAD CSV WITH HEADERS FROM 'file:///users.csv' AS row CREATE (:User {userId: row.userId, name: row.name})")
conn.query("CREATE CONSTRAINT user_constraint IF NOT EXISTS FOR (u:User) REQUIRE u.userId IS UNIQUE")

CPU times: user 3.05 ms, sys: 0 ns, total: 3.05 ms
Wall time: 2.48 s


[]

### Erstellung des CSV Files für den Import der Tweets anhand des "tweets" CSV Files 

In [9]:
tweets = pd.read_csv("tweets.csv")
f = open("import/tweetsDB.csv", "w")
writer = csv.writer(f)
writer.writerow(["tweetId","content","date_time","language",":Label"])
count = 1
for index, tweet in tweets.iterrows():
    writer.writerow([count,tweet["content"],tweet["date_time"],tweet["language"],"TWEET"])
    count += 1
f.close()

### Importieren der Tweets

In [10]:
%%time
conn.query("LOAD CSV WITH HEADERS FROM 'file:///tweetsDB.csv' AS row CREATE (:Tweet {tweetId: row.tweetId, content: row.content, date_time: row.date_time, language: row.language})")
conn.query("CREATE CONSTRAINT tweet_constraint IF NOT EXISTS FOR (t:Tweet) REQUIRE t.tweetId IS UNIQUE")

CPU times: user 3.3 ms, sys: 539 µs, total: 3.84 ms
Wall time: 1.58 s


[]

### Erstellung des CSV Files für den Import der FOLLOWS-Beziehung

In [11]:
f = open("import/follows.csv", "w")
f.write("userId,followerId,:TYPE\n")
for follow in follows:
    f.write(follow[0] + "," + follow[1] + ",FOLLOWS\n")
f.close()

### Import der FOLLOWS-Beziehungen

In [12]:
%%time
conn.query("LOAD CSV WITH HEADERS FROM 'file:///follows.csv' AS row MATCH (e:User {userId: row.userId}) MATCH (c:User {userId: row.followerId}) MERGE (e)-[:FOLLOWS]->(c)")

CPU times: user 4.06 ms, sys: 658 µs, total: 4.72 ms
Wall time: 1min 5s


[]

### Erstellung des CSV Files für den Import der POSTED-Beziehung
Jeder der Tweets wird zufällig auf einen der 100 User mit den meisten Followern verteilt.

In [13]:
top100 = conn.query("MATCH (a)-[:FOLLOWS]->(b) RETURN b, COUNT(a) as followers ORDER BY followers DESC LIMIT 100")

In [14]:
user_ids = []
for record in top100:
    user_ids.append(record.get("b").get("userId"))

tweets = pd.read_csv("import/tweetsDB.csv")
f = open("import/posted.csv", "w")
f.write("userId,tweetId,:TYPE\n")
for index, tweet in tweets.iterrows():
    f.write(f'{random.choice(user_ids)},{tweet["tweetId"]},POSTED\n')
f.close()

### Import der POSTED-Beziehungen

In [15]:
%%time
conn.query("LOAD CSV WITH HEADERS FROM 'file:///posted.csv' AS row MATCH (u:User {userId: row.userId}) MATCH (t:Tweet {tweetId: row.tweetId}) MERGE (u)-[:POSTED]->(t)")

CPU times: user 1.94 ms, sys: 252 µs, total: 2.19 ms
Wall time: 1.55 s


[]

### Erstellung des CSV Files für den Import der LIKES-Beziehung
Für jeden Tweet werden aus den Followern des Posters etwa 10 % ausgewählt, welche des Post liken. Die Like-Zahlen aus dem "tweets" CSV File werden dabei nicht berücksichtigt.

In [16]:
allTweets = conn.query("Match (t: Tweet) RETURN t");
f = open("import/likes.csv","w")
f.write("userId,tweetId,:TYPE\n")

for user in top100: 
    followers = conn.query(f"MATCH (follower:User)-[:FOLLOWS]->(following:User) WHERE following.userId ='{user['b']['userId']}' RETURN follower")
    tweets = conn.query(f"MATCH (u:User {{userId: '{user['b']['userId']}'}})-[:POSTED]->(t:Tweet) RETURN t");
    for tweet in tweets: 
        for follower in followers:
            shouldLike = random.random() < 0.1
            if shouldLike:
                f.write(f"{follower['follower']['userId']},{tweet['t']['tweetId']},LIKES\n")

f.close()    

### Import der LIKES-Beziehungen

In [17]:
%%time
conn.query("LOAD CSV WITH HEADERS FROM 'file:///likes.csv' as row CALL { with row MATCH (u:User {userId: row.userId}) MATCH (t:Tweet {tweetId: row.tweetId}) MERGE (u)-[:LIKES]->(t) }  IN TRANSACTIONS OF 100000 ROWS")

CPU times: user 11 ms, sys: 205 µs, total: 11.2 ms
Wall time: 5min 5s


[]

## Queries

### 1. Alle Posts eines zufälligen Users

In [18]:
random_poster = random.choice(top100)

result = conn.query(f"MATCH (u:User {{name: '{random_poster['b']['name']}'}})-[:POSTED]->(t:Tweet) RETURN t")
for tweet in result:
    print(tweet['t']['content'])
    print("")

#PrayingForDallas 🙏🏼😢 #PrayingForPeace

Together with the team for tomorrow's game. Hala Madrid!

Blonday #monsterstyle on Place http://t.co/fQBxFOqwCc http://t.co/cZPEEKp1wZ

When the Dalai Lama and Archbishop Desmond Tutu write a book, you gotta read it and #ShareTheJoy https://t.co/WQydH6Sr8q

@orgasmicgomez Luv that! :) #BritneyJeanOutNow

BEST joke of the year https://t.co/Z77g8WC9WC

@SuchaBandhoe I'm MORE excited that you found one of the #TheBritneyFantasy100! ;)

Day 1 that's a wrap

love you more https://t.co/hSEWcQMUHW

Ethiopia says marathoner has nothing to fear after Olympic protest. https://t.co/2cQ5X8jcMo https://t.co/zVRYyp8zKv

Throwback 2 when we were in Cuba! Can't wait to watch tonight's episode w u guys! A new episode of KUWTK starts soon https://t.co/7KCnFlJgBc

Tonight on the show: @Edward_Burns, @StevieVanZandt, music from @FLAGALine and someone let @JeffAnimalGuy back in the building. #LateNight

These four @JustinBieber fans got a surprise that you need to se

### 2. Top 100 User mit den meisten Followern

In [19]:
top100 = conn.query("MATCH (a)-[:FOLLOWS]->(b) RETURN b, COUNT(a) as followers ORDER BY followers DESC LIMIT 100")

for user in top100:
    print(user['b']['name'], "hat", user['followers'], "Follower")

user37092 hat 3383 Follower
user44740 hat 3216 Follower
user6432 hat 2735 Follower
user60693 hat 2647 Follower
user53882 hat 2471 Follower
user12527 hat 2462 Follower
user38584 hat 2133 Follower
user49269 hat 2074 Follower
user39357 hat 1905 Follower
user25599 hat 1707 Follower
user79574 hat 1632 Follower
user68562 hat 1591 Follower
user24754 hat 1521 Follower
user15250 hat 1503 Follower
user18158 hat 1500 Follower
user55950 hat 1410 Follower
user71190 hat 1402 Follower
user12729 hat 1370 Follower
user47268 hat 1337 Follower
user21840 hat 1315 Follower
user34229 hat 1273 Follower
user25044 hat 1255 Follower
user24164 hat 1231 Follower
user75352 hat 1214 Follower
user55915 hat 1201 Follower
user3575 hat 1198 Follower
user75920 hat 1186 Follower
user14617 hat 1167 Follower
user60714 hat 1154 Follower
user19411 hat 1123 Follower
user11733 hat 1120 Follower
user78789 hat 1108 Follower
user22837 hat 1098 Follower
user40933 hat 1088 Follower
user14819 hat 1072 Follower
user30997 hat 1070 Fol

### 3. Top 100 User, die den meisten Usern der Top 100 folgen

In [20]:
query = """MATCH (follower:User)-[follows:FOLLOWS]->(followed:User)
WHERE followed.userId IN [""" 

count = 0
for user in top100:
    query += f"'{user['b']['userId']}'"
    if count < 99: query += ","
    count += 1

query += """]
WITH follower, COUNT(follows) AS numFollowed
ORDER BY numFollowed DESC
RETURN follower.name AS username, numFollowed
LIMIT 100
"""

result = conn.query(query)

In [21]:
for record in result:
    print(record['username'], "folgt", record['numFollowed'], 'Usern')

user39357 folgt 69 Usern
user66702 folgt 55 Usern
user9615 folgt 50 Usern
user53146 folgt 49 Usern
user22837 folgt 47 Usern
user42799 folgt 46 Usern
user63538 folgt 44 Usern
user55559 folgt 41 Usern
user47694 folgt 40 Usern
user77717 folgt 39 Usern
user49803 folgt 38 Usern
user6684 folgt 38 Usern
user18469 folgt 37 Usern
user67633 folgt 37 Usern
user71803 folgt 37 Usern
user76680 folgt 36 Usern
user67405 folgt 35 Usern
user27072 folgt 35 Usern
user52388 folgt 35 Usern
user47722 folgt 35 Usern
user34700 folgt 34 Usern
user60305 folgt 34 Usern
user72368 folgt 34 Usern
user78827 folgt 34 Usern
user51391 folgt 34 Usern
user64787 folgt 34 Usern
user61225 folgt 33 Usern
user68562 folgt 33 Usern
user42528 folgt 33 Usern
user11975 folgt 33 Usern
user76622 folgt 31 Usern
user13415 folgt 31 Usern
user23869 folgt 31 Usern
user71776 folgt 31 Usern
user55719 folgt 31 Usern
user76576 folgt 30 Usern
user11037 folgt 30 Usern
user59719 folgt 30 Usern
user475 folgt 30 Usern
user8350 folgt 30 Usern
user5

### 4. Informationen für persönliche Startseite eines zufälligen Users (der Top 100 mit den meisten Followern)

In [22]:
random_user = random.choice(top100)

#### 1. Anzahl der Follower

In [23]:
result = conn.query(f"MATCH (follower:User)-[:FOLLOWS]->(user:User {{name: '{random_user['b']['name']}'}}) RETURN count(follower) AS followerCount")
print(result[0]['followerCount'])

1337


#### 2. Anzahl der verfolgten Accounts

In [24]:
result = conn.query(f"MATCH (user:User {{name: '{random_user['b']['name']}'}})-[:FOLLOWS]->(followed:User) RETURN count(followed) AS followedCount")
print(result[0]['followedCount'])

189


#### 3.1 Die 25 meistgeliketen Posts der verfolgten Accounts

In [25]:
result = conn.query(f"""
MATCH (follower:User)-[:FOLLOWS]->(followed:User)-[:POSTED]->(tweet:Tweet)
WHERE follower.name = '{random_user['b']['name']}'
WITH follower, collect(followed.name) AS followedUsers
MATCH (t:Tweet)<-[:POSTED]-(u:User)
WHERE u.name IN followedUsers
OPTIONAL MATCH (t)<-[:LIKES]-(liker:User)
RETURN t.content AS content, COUNT(liker) AS numLikes
ORDER BY numLikes DESC
LIMIT 25
""")

for tweet in result:
    print(tweet['content'])
    print("Likes: ", tweet['numLikes'])
    print()

Para descubrir más voces sorprendentes de la comunidad hispanohablante sigue a @InstagramES en Instagram y Twitter
Likes:  350

baby, it's cold outside
Likes:  247

Request #company
Likes:  237

i love you
Likes:  236

We've set a date! Perfume vid will premiere Tuesday on @VEVO . http://t.co/ID74wBjy91 #3DaysToPerfumeVideo
Likes:  198

'Cheek To Cheek' is now #5 on US iTunes! Keep up the good work Monsters! We're so happy. Friends just ran in screaming "CONDRAGULATIONS" !
Likes:  192

🌧🌧🌧 surreal. love you.  https://t.co/iTB1nV8Mk3
Likes:  190

Oh gosh 😳 #IAmBritneyJean
Likes:  187

YES selenagomez's photo http://t.co/af11UyAAmt
Likes:  185

A night of songwriting 💋 http://t.co/FP8BsJc22U
Likes:  184

A Tweet for those who need us, 2009 http://t.co/3Q14WKu #5yrs
Likes:  183

@WillWTH Of course! She does deserve that #1 spot
Likes:  183

Streaming LIVE in one hour:

@HillaryClinton 
#debates 
@realDonaldTrump

#GoLive on Twitter ⬇️
https://t.co/l2fPw8j3O6
Likes:  182

Head's up! Insane

#### 3.2 Die 25 neuesten Posts der verfolgten Accounts

In [26]:
result = conn.query(f"""
MATCH (follower:User)-[:FOLLOWS]->(followed:User)-[:POSTED]->(tweet:Tweet)
WHERE follower.name = '{random_user['b']['name']}'
WITH follower, collect(followed.name) AS followedUsers
MATCH (t:Tweet)<-[:POSTED]-(u:User)
WHERE u.name IN followedUsers
RETURN t.content AS content, t.date_time AS date
ORDER BY t.date_time DESC
LIMIT 25
""")

for tweet in result:
    print(tweet['content'])
    print("Getweeted am: ", tweet['date'])
    print()

Feliz 2017 de los Piqué Mebarak! Happy 2017 from the Piqué Mebarak family! https://t.co/XIayfbHnZK
Getweeted am:  31/12/2016 22:58

"Despite everything, no one can dictate who you are to other people." https://t.co/Vw9cZsFGqI
Getweeted am:  31/12/2016 21:32

OMG JUMP SCARES R US!!! 😱 😱 😱  https://t.co/ebi36sQIrw https://t.co/KkIe4TBSVw
Getweeted am:  31/12/2016 18:30

Quién ha incluido Chantaje en su playlist de @Spotify de fin de año? Nos indican dónde van a celebrarlo? https://t.co/EAtARlrG6F ShakHQ
Getweeted am:  31/12/2016 14:53

#DailyFluff
https://t.co/LUvfxwzfNl 🐾 https://t.co/nyl0tkxdOG
Getweeted am:  31/12/2016 12:03

KEEP THIS STUFF AWAY FROM CATS, MEOW! 🙀🙀🙀 https://t.co/R08apeBvyU https://t.co/8FJYoxSuBA
Getweeted am:  31/12/2016 00:30

🎿thank you to our fans and loved ones who bless us with so much each year. We had a beautiful… https://t.co/SRgCNbo2ob
Getweeted am:  31/12/2015 23:13

Ameowzing. Get it … aMEOWzing. Oh, Zing! https://t.co/7O5QKD8blT https://t.co/D0lcNw0xSa
G

#### 4. Erstellung eines neuen Posts: Herstellen einer Cashing-Beziehung bei allen Followern

In [27]:
num_tweets = conn.query("MATCH (t:Tweet) RETURN COUNT(t)")[0]['COUNT(t)']
print(num_tweets)

52542


In [28]:
tweet = {
    "tweetId": str(num_tweets+1),
    "content": "Doing neo4j queries right now #BDEA",
    "date_time": "18/06/2023 09:48",
    "language": "en"
}

##### Tweet importieren

In [29]:
new_tweet = conn.query(f"CREATE (t:Tweet {{tweetId: '{tweet['tweetId']}', content: '{tweet['content']}', date_time: '{tweet['date_time']}', language: '{tweet['language']}'}}) RETURN t")

In [30]:
print(new_tweet)

[<Record t=<Node element_id='4:267137f2-efed-4ac8-a216-d502978a187b:133848' labels=frozenset({'Tweet'}) properties={'date_time': '18/06/2023 09:48', 'language': 'en', 'tweetId': '52543', 'content': 'Doing neo4j queries right now #BDEA'}>>]


##### POSTED-Beziehung herstellen

In [31]:
conn.query(f"MATCH (user:User {{userId: '{random_user['b']['userId']}'}}) MATCH (tweet:Tweet {{tweetId: '{new_tweet[0]['t']['tweetId']}'}}) CREATE (user)-[:POSTED]->(tweet)")

[]

##### CACHED-Beziehung bei allen Followern des Posters herstellen

In [32]:
followers = conn.query(f"MATCH (follower:User)-[:FOLLOWS]->(user:User {{userId: '{random_user['b']['userId']}'}}) RETURN follower")
print(len(followers))

1337


In [33]:
print(random_user['b']['userId'])

15853668


In [34]:
for follower in followers:
    conn.query(f"MATCH (user:User {{userId: '{follower['follower']['userId']}'}}) MATCH (tweet:Tweet {{tweetId: '{new_tweet[0]['t']['tweetId']}'}}) CREATE (user)-[:CACHED]->(tweet)")

##### Check how many users have the tweet cached (should be the same as number followers)

In [35]:
conn.query("MATCH (u:User)-[r:CACHED]->(tweet:Tweet {tweetId: '52543'}) RETURN COUNT(u)")

[<Record COUNT(u)=1337>]

#### 5. Die 25 beliebtesten Tweets, welche die Wörter "hello" und "from" enthalten

In [36]:
result = conn.query("""
MATCH (t:Tweet)
WHERE ALL(word IN [' hello ', ' from '] WHERE t.content CONTAINS word)
WITH t
MATCH (t)<-[:LIKES]-(liker:User)
RETURN t.content AS content, COUNT(liker) AS numLikes
ORDER BY numLikes DESC
LIMIT 25
""")

for tweet in result:
    print(tweet['content'])
    print("Likes: ", tweet['numLikes'])
    print()

You asked: hello from France, champion ! :) im so proud of you http://t.co/AhDl5iMGzF
Likes:  216

You asked: 
Hello from Japan!
I'm Anna!I I'm a big fan! Please say me hello Anna!
 http://t.co/5Oyi7SiUhY
Likes:  179

You asked: Hi Ronaldo! I love you so much! Please say hello to me! I am from Turkey!  http://t.co/xpDH5Ppb67
Likes:  120

You asked: Can you say hello to me?  My name is Narin and I am from Germany http://t.co/lx2CRb3xwh
Likes:  114

You asked: Can you say hello to me ,please? I am from Bulgaria!!! http://t.co/dyNj55sAq4
Likes:  97

Say hello to Himanshu Yadav and all my friends from #India. I will come shortly with more news. http://t.co/60xEW67c
Likes:  93

#JoanneVibes say hello from backstage at...??? https://t.co/m9PI2oUUo8
Likes:  91

You asked: hello from morocco 
 http://t.co/gbuEyP5953
Likes:  86

Why did @Adele cross the road? She wanted to say hello from the other side.
Likes:  70

